In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
from pprint import pprint

from src.supabase_client import get_user, list_users
from tests.common import run_new_training_week_process_wrapped, run_mid_week_update_process_wrapped


# Test: One Offs

In [ ]:
# mock_user = get_user(105359460)
# response = run_new_training_week_process_wrapped(mock_user)
# print(response)

In [ ]:
# mock_user = get_user(os.environ["JAMIES_ATHLETE_ID"])
# response = run_mid_week_update_process_wrapped(mock_user)
# print(response)

# Evals: New Training Week

In [ ]:
path = 'tests/artifacts/new_training_week_errors'
files = os.listdir(path)

for file in sorted(files):
    print(f"Loading {file}")
    with open(os.path.join(path, file), "r") as f:
        results = json.load(f)
    print(sum(result['error'] for result in results) / len(results))

In [ ]:
for result in results:
    print(result['user']['email'])
    print("target:", result['recommended_weekly_mileage'])
    print("actual:", result['generated_weekly_mileage'])
    print("error:", result['error'])
    print(result['planning'])
    print("." * 100)

# Manual: New Training Week

In [13]:
from freezegun import freeze_time
from src.activities import get_activities_df, get_day_of_week_summaries, get_weekly_summaries
from src.auth_manager import get_strava_client
from src.constants import COACH_ROLE
from src.new_training_week import generate_training_week_with_coaching
from src.supabase_client import list_users

@freeze_time("2024-09-01")
def gen_training_week_wrapper(sysmsg_base, strava_client):
    activities_df = get_activities_df(strava_client)
    day_of_week_summaries = get_day_of_week_summaries(activities_df)
    weekly_summaries = get_weekly_summaries(activities_df)
    return generate_training_week_with_coaching(
        sysmsg_base=sysmsg_base,
        weekly_summaries=weekly_summaries,
        day_of_week_summaries=day_of_week_summaries,
    )

# "voynow99@gmail.com", "jaredpalek8@gmail.com", "laurenbrooks01@gmail.com", "raytrip.curtis@gmail.com"
for user in list_users():
    if user.email == "voynow99@gmail.com":
        print(user.email)
        sysmsg_base = f"{COACH_ROLE}\nYour client has included the following preferences: {user.preferences}\n"
        strava_client = get_strava_client(user.athlete_id)
        training_week_with_coaching = gen_training_week_wrapper(sysmsg_base, strava_client)
        print(training_week_with_coaching)
        break

voynow99@gmail.com
athlete_id=98390356 token still valid until 2024-09-06 04:41:50+00:00
TrainingWeekWithCoaching(typical_week_training_review=Your current training week shows a clear preference for key workouts on Wednesdays, with a moderate mileage at a pace of 9 minutes 18 seconds per mile, and long runs on Saturdays, averaging 12.65 miles at a pace of 9 minutes 31 seconds per mile. You effectively rest or have a recovery day on Sundays, and there's potential to incorporate speed work on Mondays, where you already have a moderate mileage and a slightly faster pace., weekly_mileage_target=Based on your recent training history, it seems like you've been steadily increasing your weekly mileage and long run distances. For next week, aim for a slight increase in total mileage, targeting around 46-47 miles. For your long run, focus on a range between 18-19 miles, gradually progressing from your previous long run of 18.02 miles. This slight increment will maintain your momentum while ensur

In [14]:
print(training_week_with_coaching.total_weekly_mileage)
print(training_week_with_coaching.weekly_mileage_target)
training_week_with_coaching.training_week

47.0
Based on your recent training history, it seems like you've been steadily increasing your weekly mileage and long run distances. For next week, aim for a slight increase in total mileage, targeting around 46-47 miles. For your long run, focus on a range between 18-19 miles, gradually progressing from your previous long run of 18.02 miles. This slight increment will maintain your momentum while ensuring a safe and manageable increase in training volume.


[TrainingSession(day=<Day.MON: 'mon'>, session_type=<SessionType.MODERATE: 'moderate run'>, distance=5.0, notes='Run at a moderate pace, around 8:45 per mile'),
 TrainingSession(day=<Day.TUES: 'tues'>, session_type=<SessionType.EASY: 'easy run'>, distance=3.0, notes='Run at an easy pace, around 9:18 per mile'),
 TrainingSession(day=<Day.WED: 'wed'>, session_type=<SessionType.SPEED: 'speed workout'>, distance=8.0, notes='4x1mi @ 7:30 pace with 400m recovery jog'),
 TrainingSession(day=<Day.THURS: 'thurs'>, session_type=<SessionType.EASY: 'easy run'>, distance=5.0, notes='Maintain easy pace, around 9:18 per mile'),
 TrainingSession(day=<Day.FRI: 'fri'>, session_type=<SessionType.EASY: 'easy run'>, distance=5.0, notes='Keep the pace comfortable, around 9:18 per mile'),
 TrainingSession(day=<Day.SAT: 'sat'>, session_type=<SessionType.LONG: 'long run'>, distance=18.5, notes='Maintain long run pace, around 9:31 per mile'),
 TrainingSession(day=<Day.SUN: 'sun'>, session_type=<SessionType.REST

In [15]:
# from src.supabase_client import upsert_training_week_with_coaching

# upsert_training_week_with_coaching(user.athlete_id, training_week_with_coaching)

APIResponse(data=[{'id': 73, 'athlete_id': 98390356, 'training_week': '[{"day": "mon", "session_type": "moderate run", "distance": 5.0, "notes": "Run at a moderate pace, around 8:45 per mile"}, {"day": "tues", "session_type": "easy run", "distance": 3.0, "notes": "Run at an easy pace, around 9:18 per mile"}, {"day": "wed", "session_type": "speed workout", "distance": 8.0, "notes": "4x1mi @ 7:30 pace with 400m recovery jog"}, {"day": "thurs", "session_type": "easy run", "distance": 5.0, "notes": "Maintain easy pace, around 9:18 per mile"}, {"day": "fri", "session_type": "easy run", "distance": 5.0, "notes": "Keep the pace comfortable, around 9:18 per mile"}, {"day": "sat", "session_type": "long run", "distance": 18.5, "notes": "Maintain long run pace, around 9:31 per mile"}, {"day": "sun", "session_type": "rest day", "distance": 2.5, "notes": "Active rest day, light jog or brisk walk"}]', 'typical_week_training_review': "Your current training week shows a clear preference for key workou

In [ ]:
# from src.email_manager import new_training_week_to_html, send_email
# from src.lambda_function import get_athlete_full_name


# send_email(
#     to={"email": user.email, "name": get_athlete_full_name(strava_client)},
#     subject="🏃‍♂️🎯 Just Fixed Your Jawn 🫡",
#     html_content=new_training_week_to_html(training_week_with_coaching),
# )